In [21]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.externals import joblib
import pickle

In [22]:
data = pd.read_csv('data.csv',escapechar="\\")

In [23]:
data_sub = data[['title', 'ups']]

In [24]:
# data_sub = data_sub[np.abs((data_sub.ups - data_sub.ups.mean()) <=(2*data_sub.ups.std()))]
msk = np.random.rand(len(data_sub)) < 0.8
train_set = data_sub[msk]
test_set = data_sub[~msk]
test_titles = test_set['title']
test_labels = test_set['ups']
train_titles = train_set['title']
train_labels = train_set['ups']

In [25]:
count_vectorizer = CountVectorizer(ngram_range=(1,2))
bow_transformer = count_vectorizer.fit(train_titles)
train_titles_bow = bow_transformer.transform(train_titles)
test_titles_bow = bow_transformer.transform(test_titles)

In [26]:
tfidf_transformer = TfidfTransformer().fit(train_titles_bow)
train_titles_tfidf = tfidf_transformer.transform(train_titles_bow)
test_titles_tfidf = tfidf_transformer.transform(test_titles_bow)

In [27]:
svd = TruncatedSVD()
lsa = make_pipeline(svd, Normalizer(copy=False))
X_train_lsa = lsa.fit_transform(train_titles_tfidf)
X_test_lsa = lsa.transform(test_titles_tfidf)


In [28]:
reg = Ridge(fit_intercept=False)
reg.fit(X_train_lsa, train_labels)
train_predictions = reg.predict(X_train_lsa)
test_predictions = reg.predict(X_test_lsa)
score = reg.score(X_test_lsa, test_labels)
print(score)

-0.0022917046762


In [29]:
print(train_set.iloc[0])
print(train_predictions[0], train_labels.iloc[0])

title    Mnuchin’s military flights exceed $800K
ups                                          583
Name: 1, dtype: object
651.421617415 583


In [30]:
cross_val_score(reg,X_train_lsa, train_labels, scoring='neg_mean_absolute_error')

array([-889.43694868, -900.85501907, -886.89341429])

In [31]:
pickle.dump(bow_transformer, open('bow_transformer_v1.pkl', 'wb'), protocol=2)
pickle.dump(tfidf_transformer, open('tfidf_transformer_v1.pkl','wb'), protocol=2)
pickle.dump(lsa, open('lsa_v1.pkl', 'wb'), protocol=2)
pickle.dump(reg, open('prediction_model_v1.pkl','wb'), protocol=2)

In [32]:
def apply(input):
    path1 = "bow_transformer_v1.pkl"
    path2 = "tfidf_transformer_v1.pkl"
    path3 = "lsa_v1.pkl"
    path4 = "prediction_model_v1.pkl"
    bow = pickle.load(open(path1, 'rb'))
    bow_tf = bow.transform([input])
    tfidf = pickle.load(open(path2,'rb'))
    tfidf_tf = tfidf.transform(bow_tf)
    lsa = pickle.load(open(path3,'rb'))
    lsa_tf = lsa.transform(tfidf_tf)
    model4 = pickle.load(open(path4,'rb'))
    prediction = model4.predict(lsa_tf).tolist()
    return prediction

In [33]:
apply("Donald Trump defends paper towels in Puerto Rico, says Stephen Paddock was ‘probably smart’ in bizarre TV interview: Analysis")

[551.1037753425154]